# Simulación para los smart contracts


In [ ]:
# Importación de librerias 

# Númericas
import numpy as np         # Para trabajar arreglos matriciales
import random              # Para determinar valores aleatorios
import scipy.stats as ss   # Aleatoriedad atada a una distribución de probabilidad
import math 

# Gráficas
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
import plotly.graph_objects as go

## Parámetros del protocolo

Para definir los mercados que harán parte del protocolo, se deben definir los parámetros que definirán comportamientos especificos en el modelo de *lending and borrow*, tales como el porcentaje de reservas requerido, la tasa de garantía o colateral, etc.

*  **Collateral Factor:** Es el porcentaje que se puede tomar prestado de un activo en base a lo que se suministra. Por ejemplo, supongamos que suministramos $100\text{ TK1}$ (tokens 1) al protocolo, actualmente el mercado $\text{TK1}$ cuenta con un collateral factor del $75\%$, esto nos indica que podemos tomar prestado hasta $75\text{ TK1}=100\text{ TK1}\cdot75\%$.

*  **Reserve Factor:** Es el porcentaje de la cantidad suministrada que se va a "guardar" como reservas en el protocolo.

*  **Modelo de tasas de interés:** Es la expresión matemática encargada de asignar una tasa de interés a cada mercado en función de la oferta y la demanda presentes en cada mercado.

$$\text{Borrow APR}=0.02+0.1\cdot\text{utilization rate}$$

*  **Exchange rate initial:** Es la tasa de cambio inicial entre los tokens y los cTokens. Por ejemplo, si consideramos el mercado $\text{TK1}$ y sabemos que el equivalente a $1\text{ TK1}$ en cTK1 es $0.02\text{ cTK1}$, el exchange rate actual es $0.02\text{ cTK1}$.

A continuación definiremos los parámetros de dos mercados, $\text{MK1}$ y $\text{MK2}$:

In [ ]:
# Numero de bloques por año en la red

ETHBlocks = 4*60*24*365
RSKBlocks = ETHBlocks/2

# Collateral Factors - Uno por cada mercado

collFactor_MK1 = 0.75  # Collateral factor para el mercado MK1
collFactor_MK2 = 0.75  # Collateral factor para el mercado MK2

# Lista con los Collateral Factors
collateralFactors = [collFactor_MK1,
                     collFactor_MK2]

# Reserve Factors - Uno por cada mercado

reserveFactor_MK1 = 0.20   # Reserve Factor para el mercado MK1
reserveFactor_MK2 = 0.15   # Reserve Factor para el mercado MK2
 
# Lista con los Reserve Factors                    
reserveFactors = [reserveFactor_MK1, 
                  reserveFactor_MK2]

# El modelo de tasas de interés - Una expresión para todos los mercados

def getBorrowAPR(utilizationRate):   # Borrow APR
  return 0.02 + 0.1*utilizationRate

def getSupplyAPR(borrowRate,reserveFactor,utilizationRate):  # Supply APR
  return borrowRate*utilizationRate*(1-reserveFactor)

def getAPY(rateAPR):   # APY rates
  return (1 + rateAPR/12)**12-1

## Estructura de los mercados

Dado que el estado de cada mercado se actualiza en cada bloque, debemos implementar una manera adecuada de conocer el estado del mercado y además, el estado de cada usuario (balance de la cuenta y movimientos de cada usuario). 

![texto del enlace](Imagenes/market_structure.jpg)

Cada mercado en cada bloque tiene un estado similar a la imágen anterior, cada casilla cumple una función especifica:
1.   **Borrow rate:** Es la tasa de prestamo actual, esta se calcula en base a la tasa de utilización del bloque anterior.
2.   **Supply rate:** Es la tasa de suministro actual, o de otra forma, será la tasa a la cual el lender esta ganando intereses en función de su capital suministrado. Al igual que el *borrow rate*, se calcula en base al bloque anterior.
3.   **Token price:** Es el precio actual del mercado, es una variable independiente del protocolo y viene dada por un oraculo.
4.  **Total pool reserves:** Es la cantidad de reservas actuales en el mercado.
5.  **Exchange rate:** Tasa de cambio entre tokens y ctokens en el bloque actual.
6.  **t:** Es la cantidad de bloques que lleva la tasa *borrow rate*, tambien se puede entender como la cantidad de bloques en las que no se ha hecho ninguna acción por parte de los usuarios.
7.  **Total borrow:** Es la cantidad total de tokens prestados (junto con intereses) en el mercado en el bloque actual.
8.  **Total supply:** Es la cantidad total de tokens suministrados en el mercado.
9.  **Movimiento de usuarios y registro de acciones:** En esta casilla se guardan las carteras y las acciones que realizará cada usuario en el bloque actual, cada elemento en esta casilla se puede ver como: 
![texto del enlace](Imagenes/user_structure.jpg)

Todos los usuarios que hagan parte del mercado poseen las caracteristicas o datos que se ven en el diagrama anterior: 
*   En la primera casilla encontramos la *acción* que realiza el usuario, en cada bloque cada usuario puede realizar una de cinco acciones: *supply* (o suministrar liquidez), *borrow*, *withdraw*, *repay* o no hacer nada.
*   En la segunda casilla encontramos el amount, será el monto o la cantidad de tokens que se emplean dependiendo de la acción.
*   En la tercera y cuarta posición encontramos los datos correspondientes al supply del usuario, en la tercera los montos que suministra el usuario y en la cuarta lo suministrado mas los intereses que se van acumulando.
*   Finalmente en las quinta y sexta posición encontramos los datos correspondientes a los prestamos que realice el usuario en el mercado. De manera análoga se guardan en la quinta posición los montos que se solicitan como prestamo y en la sexta posición los montos prestados mas los intereses que se generan por la deuda.


In [ ]:
# Supondremos un monto minimo y un monto máximo para el supply inicial

sup_min = 1*10**(-18)  # supply minimo inicial
sup_max = 1*10**18     # supply maximo inicial

# Tuplas de acciones de usuarios (actionIndex, Amount, supplied, supplyInterest, borrowed, borrowInterest)

# Las acciones se identificaran por los siguientes simbolos:
# s --> Supply
# b --> Borrow
# w --> Withdraw
# r --> Repay

## Configuración de un mercado

Para crear el entorno inicial debemos tener presente varios factores que influiran de manera significativa en nuestra simulación.

Lo primero es establecer la cantidad de usuarios que vamos a considerar, está debe ser la misma para todos los mercados que se quieran crear. Una vez establecido este valor debemos "llamar" los valores ya establecidos por gobernanza y podremos crear tantos mercados como necesitemos.

In [ ]:
# Formato de visualización del mercado

def fixLen(value, isNumber=False):
  if isNumber: 
    value = '%.5f' % value
    value = value[:9]
  return '{:9s}'.format(str(value))

def printMarket(market):
  blockNumber = 0
  for marketAtblock in market:
    print('accrualBlockNumber:', marketAtblock[5], '\t exchangeRate:', marketAtblock[4],
          '\t marketPrice:', marketAtblock[2])
    print('totalSupplies:', marketAtblock[7], '\t totalBorrows:', marketAtblock[6],
          '\t totalReserves:', marketAtblock[3])
    print('borrowRateAPR:', marketAtblock[0], '\t supplyRateAPR:', marketAtblock[1],'\n')
    print('|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|')
    print('| ',fixLen('# Block'),' | ',fixLen('UserIdx'),' | ',fixLen('Action'),' | ',
          fixLen('Amount'),' | ',fixLen('Supply'),' | ',fixLen('SPlusI'),' | ',
          fixLen('Borrow'),' | ',fixLen('BPlusI'),' |')    
    print('|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|')
    userIndex = 0
    for user in marketAtblock[8:]:
      print('| ',fixLen(blockNumber),' | ',fixLen(userIndex),' | ',fixLen(user[0]),' | ',fixLen(user[1], True),' | ',fixLen(user[2], True),' | ',fixLen(user[3], True),' | ',fixLen(user[4], True),' | ',fixLen(user[5], True),' |')
      userIndex += 1
    blockNumber+=1
    print('|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|','-'*11,'|')
    
MK1 = marketSetUp(5,0.2,0.5,11,0,50,getBorrowAPR_MK1)
printMarket(MK1)

In [ ]:
def getBorrowAPR_MK1(utilizationRate):   # Borrow APR MK1
  return 0.02 + 0.1*utilizationRate

def getBorrowAPR_MK2(utilizationRate):   # Borrow APR MK2
  return 0.02 + 0.1*utilizationRate

def marketSetUp(n_users, reserveFactor, initialSuppliersRate, initialTokenPrice, minSupply, maxSupply, interestModel):
  market = []  
  borrowRate = interestModel(0)
  supplyRate = getSupplyAPR(borrowRate,reserveFactor,0)
  initialExchangeRate = 50
  market.append(borrowRate)  # Borrow rate with utilizationRate = 0
  market.append(supplyRate)   # Supply rate with utilizationRate = 0
  market.append(initialTokenPrice)  # Initial Price Market
  market.append(0)  # Total Reserve
  market.append(initialExchangeRate)  # Exchange Rate
  market.append(0)  # Accrual block number
  market.append(0)  # TotalBorrow in Market
  market.append(0)  # TotalSupplied in Market
  for i in range(n_users):
    r = random.random()
    if r <= initialSuppliersRate:
      supplied = random.uniform(minSupply,maxSupply)
      userSupply = ("s",supplied,supplied,supplied*initialExchangeRate,0,0)
      market.append(userSupply)
    else:
      market.append(("x",0,0,0,0,0))
  supply = 0
  for user in market[8:]:
    supply += user[2]
  reserve = supply*reserveFactor
  market[3] = reserve  # Market reserve
  market[7] = supply   # Market total supply
  return [market]

MK1 = marketSetUp(5,0.2,0.5,11,0,50,getBorrowAPR_MK1)
printMarket(MK1)
MK2 = marketSetUp(5,0.2,0.5,5,0,5000,getBorrowAPR_MK2)
printMarket(MK2)

## Actualización de dinámicas del mercado

Dado que en cada bloque debemos conocer el estado del mercado, debemos establecer mecanismos para actualizar todas las posibles variables de cada mercado.

La manera en la que se actualizarán las tasas de interés vendrá dada por el modelo de interés y el comportamiento del mercado en el bloque anterior, esto quiere decir que en el bloque $n$ las tasas $\text{Borrow rate}$ y $\text{Supply rate}$ dependerán de la tasa de utilización (relación entre oferta y demanda) del bloque $n-1$.

Recordemos que la tasa de utilización esta dada por la siguiente ecuación:

$$\text{Utilization rate}=\frac{Total Borrow}{Total supply}\approx\frac{Total Borrow}{Total Supply+TotalBorrow}$$

De ese modo, construiremos las funciones que nos permitiran determinar los posibles cambios en las diferentes tasas del mercado en el último bloque generado:

# Stable coins
$$\text{Utilization rate}=\frac{Total Borrow}{Total Liquility + Total Borrow}$$

Borrow rate:

$$\text{Borrow rate}=\left\{\begin{array}{ll}
\text{baseBorrowRate}+\text{slope}_{1}*\text{UtilizationRate} & \text{si } utilizationRate\leq kink\\
\text{crossPoint}+\text{solpe}_{2}*\text{UtilizationRate} & \text{otherwise}\end{array}\right.$$

Supply rate:

$$\text{Supply rate}={utilization Rate*borrowRate}*({1-reserveFator})$$

In [ ]:
# Calculo de la tasa de utilización del mercado en el bloque actual

def getUtilizationRate(mk, blockNumber):
  Suppliers = []; Borrowers = []
  for user in mk[blockNumber-1][8:]:
    if user[2] != 0:  Suppliers.append(user[2])
    if user[4] != 0:  Borrowers.append(user[4])
  totalSupply = sum(Suppliers)
  totalBorrow = sum(Borrowers)
  if totalSupply == 0: return 0
  else:
    return totalBorrow/totalSupply

# Calculo de la tasa Borrow rate (por el momento se esta tomando la tasa APR)

def borrowRatePerBlock(market, blockNumber, interestModel):  
  utilizationRate = getUtilizationRate(market, blockNumber)
  borrowAPR = interestModel(utilizationRate)
  return borrowAPR

# Calculo de la tasa Supply rate (por el momento se esta tomando la tasa APR)

def supplyRatePerBlock(market, blockNumber, reserveFactor,borrowRateB):
  utilizationRate = getUtilizationRate(market, blockNumber)
  supplyAPR = getSupplyAPR(borrowRateB, reserveFactor, utilizationRate)
  return supplyAPR

UR_MK1 = getUtilizationRate(MK1,0)
BOR_MK1 = borrowRatePerBlock(MK1,0,getBorrowAPR_MK1)
SUP_MK1 = supplyRatePerBlock(MK1,0,reserveFactors[0],BOR_MK1)
print("La tasa de utilización del mercado MK1 en el bloque cero es %.4f" %UR_MK1)
print("El Borrow rate para el bloque 1 en el mercado MK1 será %.4f" %BOR_MK1)
print("El Supply rate para el bloque 1 en el mercado MK1 será %.4f" %SUP_MK1)

Para actualizar las demas variables que definen el estado del mercado en el nuevo bloque, debemos contar la cantidad de bloques en las que las tasas de interés no han sufrido cambios, es decir, bloques en los que no hubo cambios entre la oferta y la demanda del mercado. Para esto definiremos la función que cambiará el valor de $t$, el cual como ya mencionamos, será la cantidad de bloques sin cambios en las tasas de interés:

In [ ]:
# Funcion para calcular los t

def accrualBlockNumber(market, blockNumber, borrowRate):   
  if blockNumber == 0:
    return 0
  elif blockNumber == 1:
    return 1
  else:
    if market[blockNumber-1][0] != borrowRate:  # BorrowAPR[-1] y borrowAPR[-2]
      return 0
    else:
      return market[blockNumber-1][5]+1

In [ ]:
accrualBlockNumber(MK1,0)

En base al valor de $t$ se calcularán de una u otra forma los intereses tanto del mercado como de los usuarios. Analicemos esto con un ejemplo:

Supongamos que estamos interesados en realizar un suministro al mercado MK1 y que al momento de realizar el suministro las tasas de interes son $\text{Borrow rate}=0.05$ y $\text{Supply rate}=0.02$; nuestros fondos creceran con la misma tasa de interés en tanto que ningún usuario interactue con el mercado MK1, es decir, la tasa de interés será la misma mientras que ningún otro usuario (o nosotros) realicemos un supply, un borrow, un withdraw o un repay.

En este cambio el interés será del tipo **simple**, con lo cual si suponemos que en 100 bloques no se ha realizado niguna acción, tendremos

$$\text{Capital}_{100} = \text{Capital}_0*(1+100*\text{Supply rate})$$

o de manera general,

$$\text{Capital}_{\text{bloque_n}} = \text{Capital}_{\text{bloque_0}}*(1+t*\text{Supply rate})$$

si en $n$ bloques no se ha realizado ninguna acción. Por otro lado, si el valor de $t$ es cero (esto implica que algún usuario realizó una acción en el mercado MK1), se deben capitalizar los intereses, en este punto el interés se transforma en un interés del tipo compuesto, con lo cual si suponemos que en el bloque 101 alguien realizo alguna acción, tendremos

\begin{align}
\text{Capital}_{101}&=\text{Capital}_{100}*(1+\text{New supply rate})\\
&=\text{Capital}_{0}*(1+100*\text{Supply rate})*(1+\text{New supply rate})
\end{align}

Realizando así, el proceso de capitalización de intereses en el momento que se actualizan las tasas del mercado.

![texto del enlace](Imagenes/interest.jpg)

De ese modo, debemos actualziar el saldo total prestado del mercado en base a los intereses (simples o compuesto) que se calculen.

In [ ]:
# Rate por bloque
def ratePerBlock(rateAPR):
  rate = rateAPR/ETHBlocks
  return rate 

# Saldo total prestado del mercado en el bloque actual

def totalBorrowMarket(market,blockNumber):
  totalBorrow = 0
  for user in market[blockNumber-1][8:]:
    totalBorrow += user[5]
  return totalBorrow

totalBorrowMarket(MK1,0)

In [ ]:
# Saldo total suministrado en el mercado

def totalSupplyMarket(market,blockNumber):
  supply = 0
  for user in market[blockNumber-1][8:]:
    supply += user[2]   # Se toman unicamente las cantidades suministradas sin intereses
  return supply

totalSupplyMarket(MK1,0)

Continuando con la actualización de los valores propios del mercado, tenemos el total de reservas del mercado. Al igual que el saldo total prestado, las reservas tambien se capitalzian de acuerdo a la actualización de las tasas del mercado, con lo cual, la manera en la que se calcularán las reservas en un bloque $n$ será:

$$\text{TotalReserves}_n=\left\{\begin{array}{ll}
\text{TotalReserves}_{0}+\text{TotalBorrow}_{0}*(\text{BorrowRate}*t*\text{ReserveFactor}) & \text{si }t\neq0\\
\text{TotalReserves}_{n-1}+\text{TotalBorrow}_{n-1}*(\text{BorrowRate}*\text{ReserveFactor}) & \text{si }t=0\end{array}\right.$$

La expresión anterior se puede entender como que una parte (tanta como el factor de reserva) de los intereses generados por los prestamos, se irá para la reserva del mercado.

In [ ]:
# Calculo de las reservas del mercado

def totalReserveMarket(market,blockNumber,reserveFactor,borrowRatePerBlock,totalBorrow):
  if blockNumber != 0:  # Se considera desde el bloque 1 en adelante
    reserve = market[blockNumber-1][3]
    t = accrualBlockNumber(market, blockNumber,borrowRatePerBlock*(4*60*24))
    if t!=0:   # Interés simple
      reserve += totalBorrow*(borrowRatePerBlock*reserveFactor*t)  # blockNumber-1 
      return reserve  # reserve=reserv+totalBorrowmarket*(borrowAPR*t*reserveFactor)
    else:     # Interés compuesto / capitalización de intereses
      reserve += totalBorrow*(borrowRatePerBlock*reserveFactor)
      return reserve
  else:
    return market[0][3]

borrowRatePer = borrowRatePerBlock(MK1,1,getBorrowAPR_MK1)
totalBorrow = totalBorrowMarket(MK1,0)    # Debe ser del bloque anterior
totalReserveMarket(MK1,1,0.2,borrowRatePer,totalBorrow,)

Es momento de establecer el mecanismo por el cual se define la tasa de cambio entre tokens y ctokens. Para comprender esta noción debemos tener presente los siguientes conceptos:

*  **Acuñar tokens (mint tokens):** Es el proceso de suministrar algún activo en el mercado y recibir ctokens a cambio.
*  **Saldo total de subyacente:** Es la cantidad de tokens que hay en el mercado.
*  **Saldo total prestado:** Es la cantidad de tokens prestados.
*  **Reservas:** Reservas actuales del mercado.

Con lo cual, la expresión que define la tasa de cambio en el bloque actual es

$$TokenPrice = \frac{1}{cTokenPrice}=\frac{TotalUnderlyingBalance+TotalBorrowBalance-Reserves}{cTokenMinted}$$

Una característica que cabe mencionar es que a pesar de que la tasa de cambio depende del saldo total suministrado y si este baja, la tasa de cambio tambien podrá bajar, el crecimiento en el valor de los cTokens se verá directamente relacionado con la acumulación de intereses generados por los prestamos de cada usuario.

Para determinar la cantidad $cTokenMinted$ debemos tomar la cantidad de tokens en el mercado (incluyendo los tokens que fueron prestados fuera de intereses) y dividirlos por el precio anterior de los cTokens.

In [ ]:
# Tasa de cambio para un mercado

def exchangeRateMarket(market,blockNumber,reserveFactor,totalSupply,totalBorrow,totalReserves):
  if blockNumber == 0:
    return 50   # exchangeRate inicial
  else:
    if market[blockNumber-1][4] == 0:
      exchangeRate = market[0][4]
    else:
      exchangeRate = market[blockNumber-1][4]
    #cTokenMinted = totalSupply*exchangeRate   # Cambio a precios muy elevados del ctoken
    cTokenMinted = 0
    for user in market[blockNumber-1][8:]:
      cTokenMinted += user[3]
    if totalSupply+totalBorrow-totalReserves != 0:
      exchangeRate = cTokenMinted/(totalSupply+totalBorrow-totalReserves)
    return exchangeRate

Una vez obtenidos todos los valores anteriormente mencionados, podemos definir el estado del mercado en el siguiente bloque; es importante mencionar que en este punto aun no hemos tomado en cuenta los movimientos de los usuarios.

In [ ]:
# Definición del estado del mercado en el siguiente bloque 
def setMarketVariablesInBlock(market, blockNumber, reserveFactor, marketPrice, interestModel):  # blockNumber -1
  borrowRate = borrowRatePerBlock(market,blockNumber,interestModel)
  supplyRate = supplyRatePerBlock(market,blockNumber,reserveFactor,borrowRate)
  totalBorrow = totalBorrowMarket(market,blockNumber)  #-1
  totalReserve = totalReserveMarket(market,blockNumber,reserveFactor,borrowRate,totalBorrow)
  totalSupply = totalSupplyMarket(market,blockNumber)  #-1
  exchangeRate = exchangeRateMarket(market,blockNumber,reserveFactor,totalSupply,totalBorrow,totalReserve)
  t = accrualBlockNumber(market,blockNumber,borrowRate)
  mkAtBlock = [('x', 0, 0, 0, 0, 0)] * len(market[0])
  if exchangeRate == 0:
    exchangeRate = market[0][2]
  mkAtBlock[0] = borrowRate
  mkAtBlock[1] = supplyRate
  mkAtBlock[2] = marketPrice
  mkAtBlock[3] = totalReserve
  mkAtBlock[4] = exchangeRate
  mkAtBlock[5] = t 
  mkAtBlock[6] = totalBorrow
  mkAtBlock[7] = totalSupply
  userIndex = 8 #                                               Anni did this
  for users in market[blockNumber-1][8:]: #                     Anni did this
    mkAtBlock[userIndex] = market[blockNumber-1][userIndex] #   Anni did this
    userIndex += 1 #                                            Anni did this
  return mkAtBlock

MK1_inBlock1 = setMarketVariablesInBlock(MK1,0,reserveFactors[0],12,getBorrowAPR_MK1)
#printMarket(MK1)
#Actualización de variables del mercado MK1 para el siguiente bloque
printMarket([MK1_inBlock1])

## Update de los movimientos de los usuarios

Para simular cada una de las acciones que pueden realizar los usuarios será necesario implementar una distribución de probabilidad que modele la cantidad promedio de ocurrencias de algún suceso en un tiempo especifico, en nuestro contexto se traduce por ejemplo, a la cantidad promedio de usuarios que hacen supply en un bloque.

La distribución de probabilidad la podemos entender como una expresión matemática que asigna a cada valor, la probabilidad de que "aparezca" dicho valor. 

En ese sentido, la distribución de probabilidad que mejor se ajusta a nuestra simulación es la distribución de Pöisson, la cual nos indicara la probabilidad de que uno o dos, o 100 usuarios realicen supply (por ejemplo) en un bloque dado.

Para comprender mejor la noción de distribución de probabilidad que vamos a implementar en la simulación realizaremos un ejemplo gráfico.

Supongamos que las condiciones del mercado son tales que el promedio de entradas de suppliers por bloque es 5, la cantidad de borrowers promedio es de 3, la de usuarios que realizan withdraw es de 1 y los que hacen un repay son 2 en promedio. Las distribuciones de probabilidad que se ajustan a estos valores son:


In [ ]:
# Valores estimados de ocerrencias

lambda_suppliers = 5    # Candidad estimada de llegada de suppliers por bloque  
lambda_borrowers = 3    # Candidad estimada de llegada de borrowes por bloque
lambda_withdraws = 1    # Candidad estimada de usuarios que realizan withdraw por bloque
lambda_repays = 2       # Candidad estimada de usuarios que realizan repay por bloque

# Distribuciones de Poisson para los valores dados

suppliersDistribuion = ss.poisson(lambda_suppliers)   
borrowersDistribuion = ss.poisson(lambda_borrowers)   
withdrawsDistribuion = ss.poisson(lambda_withdraws)   
repaysDistribuion = ss.poisson(lambda_repays)

# Visualización

people = np.arange(15)

suppliersProbability = []  
borrowersProbability = []
withdrawsProbability = []
repaysProbability = []

for user in people:
  suppliersProbability.append(suppliersDistribuion.pmf(user))   
  borrowersProbability.append(borrowersDistribuion.pmf(user))   
  withdrawsProbability.append(withdrawsDistribuion.pmf(user))
  repaysProbability.append(repaysDistribuion.pmf(user))

# Visualización
fig = go.Figure()

fig.add_trace(go.Scatter(x=people,y=suppliersProbability,name="Suppliers"))
fig.add_trace(go.Scatter(x=people,y=borrowersProbability,name="Borrowers"))
fig.add_trace(go.Scatter(x=people,y=withdrawsProbability,name="Withdraws"))
fig.add_trace(go.Scatter(x=people,y=repaysProbability,name="Repays"))

fig.update_layout(title="Distribuciones para la acción por parte de cada usuario por bloque",
                  xaxis_title='Cantidad de personas',
                  yaxis_title='Probabilidad')
fig.show()

En la gráfica anterior podemos ver que la probabilidad de que 4 usuarios hagan supply es mayor por ejemplo a la probabilidad de que 4 usuarios hagan withdraw, también podemos ver que es mucho mas probable que 2 usuario hagan repay a que 7 usuarios lo hagan. Esta será la idea que implementaremos para modelar la ocurrencia de cada acción.

A continuación definiremos cada uno de los posibles eventos que se pueden presentar en cada bloque:

1.   **Nuevos suppliers**



![texto del enlace](Imagenes/supply.jpg)

In [ ]:
def temporalBlock(market,blockNumber):
  temporal = []
  for value in market[blockNumber][:8]:
    temporal.append(value)
  for user in market[blockNumber][8:]:
    temporal.append(("x",0,0,0,0,0))
  return temporal

In [ ]:
def newSuppliers(market,estimatedOccurrences,blockNumber,minSupply,maxSupply,restriction=True):
  marketCopyB = []
  temporal = temporalBlock(market,blockNumber)
  for user in market[blockNumber]:
    marketCopyB.append(user)
  poissonDistribution = ss.poisson(estimatedOccurrences)
  n_users = len(marketCopyB[8:])
  numberOfPeople = np.arange(n_users)
  probability = []
  for i in numberOfPeople:
    probability.append(poissonDistribution.pmf(i))
  numberOfNewSuppliers = random.choices(numberOfPeople,probability)[0]
  users = []
  userIndex = 8
  for user in marketCopyB[8:]:
    users.append((user,userIndex))
    userIndex += 1
  for newSupplier in range(numberOfNewSuppliers):
    user, userIndex = random.sample(users,1)[0]
    users.remove((user, userIndex))
    if restriction == True:
      if user[5] == 0:   # Si no tiene prestamos pendientes en este mercado
        supplied = random.uniform(minSupply,maxSupply)
        user = ("s", supplied, 0, 0, 0, 0)
        temporal[userIndex]=user
    else:
      supplied = random.uniform(minSupply,maxSupply)
      user = ("s", supplied, 0 , 0, 0, 0)
      temporal[userIndex]=user
  return temporal

MK1_NS = newSuppliers(MK1,2,0,0,55)
printMarket([MK1_NS])

2.  **Nuevos borrowers**

![texto del enlace](Imagenes/borrow.jpg)

Para determinar la capacidad de prestamo de cada usuario, se deben tener en cuenta los movimientos del mismo usuario en todos los mercados, el siguiente diagrama nos permite determinar el valor de la capacidad de prestamo:

![texto del enlace](Imagenes/borrow_limit.jpg)

Las casillas que vemos en el paso intermedio del diagrama anterior, corresponde con los balances del usuario en cada mercado, al realizar el dichos calculos se suman los valores obtenidos y el resultado será la capacidad de préstamo del usuario.

In [ ]:
# Capacidad de prestamo del usuario

def borrowlimit(markets,blockNumber,userIndex,collateralFactors):
  capacity = 0
  for m in range(len(markets)):
    user = markets[m][blockNumber][userIndex]
    capacity += (user[2]-user[5])*collateralFactors[m]*markets[m][blockNumber][2]
  return capacity   # Capacidad de prestamo en USD

# Nuevos borrowers

# marketBor --> Mercado donde se va a pedir prestado
def newBorrowers(markets,marketBor,estimatedOccurrences,blockNumber,collateralFactors):
  marketCopyB = []
  temporal = temporalBlock(marketBor,blockNumber)
  for user in marketBor[blockNumber]:
    marketCopyB.append(user)
  poissonDistribution = ss.poisson(estimatedOccurrences)
  n_users = len(marketCopyB[8:])
  numberOfPeople = np.arange(n_users)
  probability = []
  for i in numberOfPeople:
    probability.append(poissonDistribution.pmf(i))
  numberOfNewBorrowers = random.choices(numberOfPeople,probability)[0]
  users = []
  userIndex = 8
  for user in marketCopyB[8:]:
    users.append((user,userIndex))
    userIndex += 1
  for newBorrower in range(numberOfNewBorrowers):
    user, userIndex = random.sample(users,1)[0]
    users.remove((user, userIndex))
    liquidity = marketBor[blockNumber][7]-marketBor[blockNumber][6]-marketBor[blockNumber][3]
    limit = borrowlimit(markets,blockNumber,userIndex,collateralFactors)/marketBor[blockNumber][2]
    limit = min(limit,liquidity)
    if limit > 0:
      borrowed = random.uniform(0,limit)
      user = ("b", borrowed, 0, 0, 0, 0)  # Se añade bloque donde pidio, monto, pagos 
      temporal[userIndex]=user                                           
  return temporal

markets = [MK1,MK2]
updateBorrowers = newBorrowers(markets,MK2,3,0,[0.2,0.2])
printMarket([updateBorrowers])

In [ ]:
printMarket(MK1)
printMarket(MK2)

3.   **Nuevos withdraws**

![texto del enlace](Imagenes/withdraw.jpg)

En el diagráma anterior aparecen dos nuevos conceptos que no habiamos mencionado, el **ratio** del usuario y la **liquidez del mercado**.

Recordemos que el collateral factor nos indicaba el porcentaje de colateral que podiamos tomar prestado, el ratio es el reciproco del collateral factor, es decir, el ratio nos permite determinar cual es la cantidad de collateral que necesitamos para dar garantia sobre el prestamo que tenemos.

Por ejemplo, supongamos que tenemos un deposito de 100 tokens en el mercado $MK1$, el cual cuenta con un collateral factor de $75\%$, esto nos indica que podemos realizar un prestamo de hasta 75 tokens en el mercado. Supongamos que realizamos dicho prestamo y que posteriormente pagamos 50 tokens de la deuda, quedandonos asi un saldo de 25 tokens en el prestamo (para el ejemplo no tomaremos en cuenta los intereses), la pregunta sería que tanto podemos sacar del colateral que teniamos? La respuesta nos la da el ratio y se calcula de la siguiente manera

\begin{align}
ratio &= \frac{1}{collateralFactor}*borrowBalance \\
&=\frac{1}{75\%}*25\\
&=33.3
\end{align}

De ese modo podremos realizar un withdraw de hasta $66.6\text{ tokens} = 100-33.3$.

Por otro lado tenemos la liquidez del mercado, la cual en terminos concretos es la cantidad de tokens que hay en el mercado, incluyendo las reservas. Cabe resaltar que si se retiran tokens de la reserva se esta adquiriendo riesgo de liquidez en el mercado. 

La manera en la que se calcula la liquidez del mercado es 

$$\text{liquidity}=\text{total supply}-\text{total borrow balance}$$



In [ ]:
def marketLiquidity(market,blockNumber):  
  supply = totalSupplyMarket(market, blockNumber)
  borrow = totalBorrowMarket(market,blockNumber)
  return supply-borrow

# Limite del withdraw

# marketWithdraw --> Mercado donde se va a hacer el withdraw
def withdrawLimitMarket(markets,marketWithdraw,blockNumber,collateralFactors,userIndex):
  ratio = 0
  for m in range(len(markets)):
    supplyBalance = markets[m][blockNumber][userIndex][3]*markets[m][blockNumber][4]
    borrowBalance = markets[m][blockNumber][userIndex][5]
    ratio += ((supplyBalance - borrowBalance)/collateralFactors[m])*markets[m][blockNumber][2]
  suppliedMarket = marketWithdraw[blockNumber][userIndex][3]/marketWithdraw[blockNumber][4]
  liquidity = marketLiquidity(marketWithdraw,blockNumber)*markets[m][blockNumber][2]
  return min(ratio,suppliedMarket,liquidity)

# Nuevos Withdraws
def newWithdraws(markets,marketWith,estimatedOccurrences,blockNumber,collateralFactors):
  marketCopyB = []
  temporal = temporalBlock(marketWith,blockNumber)
  suppliers = []
  for user in marketWith[blockNumber]:
    marketCopyB.append(user)
  poissonDistribution = ss.poisson(estimatedOccurrences)
  npeople = 0
  userIndex = 8
  for user in marketCopyB[8:]:
    if user[3] != 0:
      npeople += 1
      suppliers.append((user,userIndex))
    userIndex += 1
  numberOfPeople = np.arange(npeople+1)
  probability = []
  for i in numberOfPeople:
    probability.append(poissonDistribution.pmf(i))
  numberOfNewWithdraws = random.choices(numberOfPeople,probability)[0]
  users = []
  userIndex = 8
  for user in marketCopyB[8:]:
    users.append((user,userIndex))
    userIndex += 1
  numberOfNewWithdraws = min(numberOfNewWithdraws,len(suppliers))
  for newWith in range(numberOfNewWithdraws):
    user, userIndex = random.sample(users,1)[0]
    users.remove((user, userIndex))
    withdrawLimit = withdrawLimitMarket(markets,marketWith,blockNumber,collateralFactors,userIndex)
    if withdrawLimit > 0:
      withdrawAmount = (random.uniform(0,withdrawLimit))/marketWith[blockNumber][2]
      user = ("w",withdrawAmount,0,0,0,0)
    else:
      user = ("x",0,0,0,0,0)
    temporal[userIndex]=user
  return temporal

newWith = newWithdraws(markets,MK1,3,0,[0.2,0.2])
printMarket([newWith])

4.  **Nuevos repays**

![texto del enlace](Imagenes/repay.jpg)

In [ ]:
def newRepays(market, estimatedOccurrences, blockNumber):
  marketCopyB = []
  temporal = temporalBlock(market,blockNumber)
  for user in market[blockNumber]:
    marketCopyB.append(user)
  poissonDistribution = ss.poisson(estimatedOccurrences)
  borrowers = []
  npeople = 0
  userIndex = 8
  for user in marketCopyB[8:]:
    if user[5] != 0:  
      npeople += 1
      borrowers.append((user,userIndex))
    userIndex += 1
  numberOfPeople = np.arange(npeople+1)
  probability = []
  for i in numberOfPeople:
    probability.append(poissonDistribution.pmf(i))
  numberOfNewRepaies = random.choices(numberOfPeople,probability)[0]
  users = []
  userIndex = 8
  for user in marketCopyB[8:]:
    users.append((user,userIndex))
    userIndex += 1
  for newRepay in range(numberOfNewRepaies):
    user, userIndex = random.sample(users,1)[0]
    users.remove((user, userIndex))
    repay = random.uniform(0,user[5]) 
    if repay != 0:
      user = ("r",repay,0,0,user[5]-repay,user[5]-repay)
    else:
      user = ("x",0,0,0,user[5],user[5])
    temporal[userIndex]=user
  return temporal

In [ ]:
def newNothings(market, blockNumber):
  marketCopyB = []
  temporal = temporalBlock(market,blockNumber)
  for user in market[blockNumber]:
    marketCopyB.append(user)
  userIndex = 8
  for user in marketCopyB[8:]:
    user = ("x",0,0,0,0,0)
    temporal[userIndex] = user
    userIndex += 1
  return temporal

## Generación del nuevo bloque

In [ ]:
def interestSupply(market,blockNumber,borrowRate):
  marketCopy = []
  t = accrualBlockNumber(market,blockNumber,borrowRate)
  currentExchangeRate = market[blockNumber][4]
  for element in market[blockNumber]:
    marketCopy.append(element)
  for user in marketCopy[8:]:
    userIndex = marketCopy.index(user)
    if t != 0:
      user = (user[0],user[1],user[2],user[3],user[4],user[5])   # No hay capitalizacion
    else:
      #capitalization = user[2]*currentExchangeRate
      capitalization = user[2]
      user = (user[0],user[1],capitalization,user[3],user[4],user[5])   # Capitalizacion
    marketCopy[userIndex] = user
  return marketCopy

def interestBorrow(market,blockNumber,borrowRate):
  marketCopy = []
  t = accrualBlockNumber(market,blockNumber,borrowRate)
  for element in market[blockNumber]:
    marketCopy.append(element)
  for user in marketCopy[8:]:
    userIndex = marketCopy.index(user)
    if t != 0:
      interest = user[4]*(1+t*market[blockNumber][0])
      user = (user[0],user[1],user[2],user[3],user[4],interest)  # No hay capitalizacion
    else:
      interest = user[5]*(1+market[blockNumber][0])
      user = (user[0],user[1],user[2],user[3],user[5],interest)
    marketCopy[userIndex] = user
  return marketCopy

def updateInterest(market,blockNumber,borrowRate):
  supplyInterests = interestSupply(market,blockNumber,borrowRate)
  borrowInterests = interestBorrow(market,blockNumber,borrowRate)
  marketCopy = []
  for value in market[blockNumber-1]:
    marketCopy.append(value)
  if blockNumber > 0:
    userIndex = 8
    for user in marketCopy[8:]:
      supply = supplyInterests[userIndex][2]
      supplyInterest = supplyInterests[userIndex][3]
      borrow = borrowInterests[userIndex][4]
      borrowInterest = borrowInterests[userIndex][5]
      user = ("x",0,supply,supplyInterest,borrow,borrowInterest)
      marketCopy[userIndex] = user
      userIndex += 1
    return marketCopy
  else:
    return marketCopy

interestUpdate = updateInterest(MK1,0,0.023)
#printMarket(MK1)
printMarket([interestUpdate])

In [ ]:
liquidationIncentive = 0.08
closeFactor = 0.05

def borrowPositions(markets,blockNumber,userIndex):
  positions = []
  for market in markets:
    positions.append(market[blockNumber][userIndex])
  return positions

def totalBorrowUser(markets, blockNumber, positions, userIndex):
  borrow = 0
  for p in range(len(markets)):
    borrow += positions[p][5]*markets[p][blockNumber][2]
  return borrow  # En USD

def liquidationIndicator(markets,blockNumber,positions,collateralFactors,userIndex):
  userRatio = 0   # Valor en USD
  marketsRatio = 0
  for p in range(len(positions)):
    if positions[p][5] != 0:
      userRatio += (positions[p][3]/markets[p][blockNumber][4])*markets[p][blockNumber][2]/positions[p][5]
    else:
      userRatio += (positions[p][3]/markets[p][blockNumber][4])*markets[p][blockNumber][2]
    marketsRatio += markets[p][blockNumber][2]/collateralFactors[p]
  if userRatio < marketsRatio:
    return True   # Si se cumple, la cuentra entra en liquidación
  else: 
    return False

def payLoanPerMarket(pay,market,blockNumber,borrowPosition,userIndex):
  PositionCopy = []
  for value in borrowPosition:
    PositionCopy.append(value)
  amount = pay/market[blockNumber][2]   # Valor del pago en tokens del mercado
  balance = PositionCopy[5] - amount
  if balance >= 0:
    user = ("x",0,PositionCopy[2],PositionCopy[3],balance,balance)
    pay = 0  # Se gasto toda la plata
  else:
    user = ("x",0,PositionCopy[2],PositionCopy[3],0,0)
    pay = abs(balance)*market[blockNumber][2]   # Valor actualizado del pago en USD
  PositionCopy = user
  return [pay, PositionCopy]

def payPerLiquidation(payPerLiq,market,blockNumber,collateralPosition,userIndex):
  PositionCopy = []
  for value in collateralPosition:
    PositionCopy.append(value)
  amount = payPerLiq/market[blockNumber][2]   # Valor del pago en tokens del mercado
  balance = PositionCopy[3]/market[blockNumber][4] - amount
  if balance >= 0:
    user = ("x",0,balance,balance*market[blockNumber][4],PositionCopy[4],PositionCopy[5])
    payPerLiq = 0  # Se gasto toda la plata
  else:
    user = ("x",0,0,0,PositionCopy[4],PositionCopy[5])
    payPerLiq = abs(balance)*market[blockNumber][2]   # Valor actualizado del pago en USD
  PositionCopy = user
  return [payPerLiq, PositionCopy]

def newLiquidator(markets,blockNumber,positions,pay,userIndex,liquidationIncentive=liquidationIncentive):
  totalPay = pay
  payPerLiq = totalPay*(1+liquidationIncentive)
  for p in range(len(positions)):
    newposition = payPerLiquidation(payPerLiq,markets[p],blockNumber,positions[p],userIndex)
    payPerLiq = newposition[0]
    positions[p] = newposition[1]
  positions = positions[::-1]
  markets = markets[::-1]
  for p in range(len(positions)):
    newposition = payLoanPerMarket(pay,markets[p],blockNumber,positions[p],userIndex)
    pay = newposition[0]
    positions[p] = newposition[1]
  positions = positions[::-1]
  markets = markets[::-1]
  return positions

def liquidation(markets,blockNumber,userIndex,collateralFactors,liquidationIncentive=liquidationIncentive,closeFactor=closeFactor):
  positions = borrowPositions(markets,blockNumber,userIndex)
  indicator = liquidationIndicator(markets,blockNumber,positions,collateralFactors,userIndex)
  totalPay = 0
  if indicator == True:
    borrow = totalBorrowUser(markets,blockNumber,positions,userIndex)
    totalBorrow = borrow
    k = 0
    while indicator == True and borrow != 0 and k < 100:
      pay = totalBorrow*closeFactor   # Paga tanto como el close factor le permita
      totalPay += pay
      newpositions = newLiquidator(markets,blockNumber,positions,pay,userIndex,liquidationIncentive)
      for p in range(len(positions)):
        positions[p] = newpositions[p]
      indicator = liquidationIndicator(markets,blockNumber,positions,collateralFactors,userIndex)
      borrow = totalBorrowUser(markets,blockNumber,positions,userIndex)
      k += 1
    for p in range(len(positions)):
      position = (positions[p][0]+"l",totalPay,positions[p][2],positions[p][3],positions[p][4],positions[p][5])
      positions[p] = position
    return positions
  else:
    return positions

  #def reserveModification(payPerLiq,markets,blockNumber):
  #for market in markets:
  #  if payPerLiq > 0:
  #    payPerLiq = payPerLiq - market[blockNumber][3]
  #  else:
  #     market[blockNumber][3] = abs(payPerLiq)
  #     break
  #return markets

userliq = ("x",0,10,500,4,6)
marketLiq = [[0.25,0.02,10,5,50,1,25,10,userliq,("x",0,10,500,2,4)]]
market2 =[[0.2,0.1,5,15,45,0,0,50,("x",0,5,225,10,13),("x",0,10,450,2,4)]]
markets = [marketLiq,market2]
#positionsUser = borrowPositions(markets,0,8)
#print(positionsUser)
#print(newLiquidator(markets,0,22.06492262371665,8))
#print(liquidation(markets,0,9,[0.75,0.75]))
liquidation(markets,0,0+8,[0.75,0.75])
#positions = borrowPositions(markets,1,8)
#print(positions)
#liquidationIndicator(markets,0,positionsUser,[0.75,0.75],8)
#newLiquidator(markets,0,positionsUser,50,8)

In [ ]:
TRU_supply = [6000,1000,5000,6000,1000,8000,1000,1500,2000,2000]
TRU_borrow = [2000,1500,8000,1000,500,5000,300,500,1000,2000]
TRU_withdraw = [1000,500,2000,450,1500,200,250,1000,2000,200]
TRU_repays = [150,900,500,800,200,500,250,1000,1000,1000]
suppliersPerDay = sum(TRU_supply)/len(TRU_supply)
suppliersPerBlock = suppliersPerDay/(4*60*24)
borrowersPerDay = sum(TRU_borrow)/len(TRU_borrow)
borrowersPerBlock = borrowersPerDay/(4*60*24)
withdrwasPerDay = sum(TRU_withdraw)/len(TRU_withdraw)
withdrwasPerBlock = withdrwasPerDay/(4*60*24)
repaysPerDay = sum(TRU_repays)/len(TRU_repays)
repaysPerBlock = repaysPerDay/(4*60*24)
#people = int(suppliersPerBlock+borrowersPerBlock+withdrwasPerBlock+repaysPerBlock)

print(suppliersPerBlock, borrowersPerBlock,withdrwasPerBlock,repaysPerBlock)
#print(suppliersPerBlock/people,borrowersPerBlock/people,withdrwasPerBlock/people,repaysPerBlock/people)

In [ ]:
def preferences(market,blockNumber):
  initialPriceMarket = market[0][2]


In [ ]:
def implications(market,blockNumber):
  withdraws = 0
  for user in market[blockNumber][8:]:
    if user[0] == "w":
      withdraws += user[1]
  cash = market[blockNumber][7] - market[blockNumber][6] - market[blockNumber][3]
  balanceMarket = cash - withdraws
  if balanceMarket > 0:
    return market
  elif balanceMarket <= 0:
    market[blockNumber][3] = max(0,market[blockNumber][3] - abs(balanceMarket))
    return market   

In [ ]:
# Valores para los lambda
suppliersPerBlock = suppliersPerBlock
borrowersPerBlock = borrowersPerBlock
withdrwasPerBlock = withdrwasPerBlock
repaysPerBlock = repaysPerBlock

def newBlockMarket(markets,market,collateralFactors,blockNumber,reserveFactor,priceMarket,interestModel,minSupply,maxSupply,restriction=False):
  marketCopy = []
  for value in market[blockNumber-1]:
    marketCopy.append(value)  # copia del mercado
  blockMarket = setMarketVariablesInBlock(market,blockNumber,reserveFactor,priceMarket,interestModel)  # blockNumber-1
  market.append(blockMarket)
  newSup = newSuppliers(market,suppliersPerBlock,blockNumber-1,minSupply,maxSupply,restriction)  # suppliersPerBlock
  newBor = newBorrowers(markets,market,borrowersPerBlock,blockNumber-1,collateralFactors)  # borrowersPerBlock
  newWith = newWithdraws(markets,market,withdrwasPerBlock,blockNumber-1,collateralFactors)  # withdrwasPerBlock
  newRep = newRepays(market,repaysPerBlock,blockNumber-1)  # repaysPerBlock
  newNot = newNothings(market,blockNumber-1)
  updateInterestBlock = updateInterest(market,blockNumber-1,blockMarket[0])   # Hasta aqui se toman las acciones
  userIndex = 8
  for user in marketCopy[8:]:
    actions = [newSup[userIndex],newBor[userIndex],newWith[userIndex],newRep[userIndex],newNot[userIndex]]
    action = random.sample(actions,1)[0] 
    interests = updateInterestBlock[userIndex]
    if action[0] == "s":
      user = (action[0],action[1],(interests[3]/market[blockNumber][4])+action[1],interests[3]+(action[1]*market[blockNumber][4]),interests[4],interests[5])
    elif action[0] == "b":
      user = (action[0],action[1],interests[2],interests[3],interests[4]+action[1],interests[5]+action[1])
    elif action[0] == "w":
      if action[1] <= interests[3]/market[blockNumber][4]:
        user = (action[0],action[1],(interests[3]/market[blockNumber][4])-action[1],interests[3]-(action[1]*market[blockNumber][4]),interests[4],interests[5])
      else:
        user = (action[0],interests[3]/market[blockNumber][4],0,0,interests[4],interests[5])
    elif action[0] == "r":
      user = (action[0],action[1],interests[2],interests[3],action[4],action[5])
    else:   
      user = (action[0],action[1],interests[2],interests[3],interests[4],interests[5])
    market[blockNumber][userIndex] = user
    userIndex += 1
  market = implications(market,blockNumber)
  return market

MK1 = marketSetUp(5,0.2,0.5,10,0,50,getBorrowAPR_MK1)
MK2 = marketSetUp(5,0.15,0.8,5,0,20,getBorrowAPR_MK2)
markets = [MK1,MK2]
#printMarket(MK1)
markets = [MK1,MK2] 
collateralFactors = [0.75,0.75]
reserveFactors = [0.2,0.15]
priceMarkets = [11,4]

MK1_1 = newBlockMarket(markets,MK1,collateralFactors,1,0.2,11,getBorrowAPR_MK1,0,5000)
MK2_1 = newBlockMarket(markets,MK2,collateralFactors,1,0.2,11,getBorrowAPR_MK2,0,200)

MK1_2 = newBlockMarket(markets,MK1,collateralFactors,2,0.2,13,getBorrowAPR_MK1,0,5000)
MK2_2 = newBlockMarket(markets,MK2,collateralFactors,2,0.2,15,getBorrowAPR_MK2,0,200)

#MK1_3 = newBlockMarket(markets,MK1,collateralFactors,3,0.2,18,getBorrowAPR_MK1,0,50000)
#MK2_3 = newBlockMarket(markets,MK2,collateralFactors,3,0.2,12,getBorrowAPR_MK2,0,2000)

#print('\n=================================== Market 1 ===================================\n')
printMarket(MK1)

print('\n=================================== Market 2 ===================================\n')
printMarket(MK2)

In [ ]:
def newStateMarkets(markets,collateralFactors,blockNumber,reserveFactors,priceMarkets,interestModels,minSupplyPerMarkets,maxSupplyPerMarkets,restriction=True,liquidationIncentive=liquidationIncentive,closeFactor=closeFactor):
  for m in range(len(markets)):
    markets[m] = newBlockMarket(markets,markets[m],collateralFactors,blockNumber,reserveFactors[m],priceMarkets[blockNumber][m],interestModels[m],minSupplyPerMarkets[m],maxSupplyPerMarkets[m],restriction)
  for userIndex in range(len(markets[0][blockNumber][8:])):
    liquidations = liquidation(markets,blockNumber,userIndex+8,collateralFactors,liquidationIncentive,closeFactor)
    for p in range(len(markets)):
      markets[p][blockNumber][userIndex+8] = liquidations[p]
  return markets

userliq = ("x",0,10,500,4,6)
marketLiq = [[0.25,0.02,10,5,50,1,25,10,userliq,("x",0,10,500,2,4)]]
market2 =[[0.2,0.1,5,15,45,0,0,50,("x",0,5,225,10,13),("x",0,10,450,2,4)]]
markets = [marketLiq,market2]
newState = newStateMarkets(markets,[0.75,0.75],1,[0.2,0.15],[[0,0],[10,5],[10,5]],[getBorrowAPR_MK1,getBorrowAPR_MK2],[0,0],[0,0])
markets = newState
newState = newStateMarkets(markets,[0.75,0.75],2,[0.2,0.15],[[0,0],[10,5],[10,5]],[getBorrowAPR_MK1,getBorrowAPR_MK2],[0,0],[0,0])
printMarket(newState[0])
print("---------------------------------------")
printMarket(newState[1])

In [ ]:
from IPython.core.pylabtools import figsize
figsize(16,6)

In [ ]:
def preutilization(borrowRate):
  return (borrowRate - 0.02)/0.1

In [ ]:
n_users = 5
MK1 = marketSetUp(n_users,0.2,0.7,5,0,50,getBorrowAPR_MK1)
MK2 = marketSetUp(n_users,0.15,0.5,10,0,20,getBorrowAPR_MK2)
markets = [MK1,MK2]

n_iterations =  50 #4*60*24
for i in range(n_iterations):
  MK1 = newBlockMarket(markets,MK1,collateralFactors,i+1,0.2,5,getBorrowAPR_MK1,0,50)
  MK2 = newBlockMarket(markets,MK2,collateralFactors,i+1,0.2,12,getBorrowAPR_MK2,0,20)
  if MK1[i+1][4]<0 or MK2[i+1][4]<0:
    print("Desde aquí se jodio la cosa", i)
    break
  markets = [MK1,MK2]

# plot supply rate
ur1 = []
ur2 = []
for i in range(len(MK1)):
  ur_MK1 = preutilization(MK1[i][0])
  ur_MK2 = preutilization(MK2[i][0])
  ur1.append(ur_MK1)
  ur2.append(ur_MK2)

# Borrow APR 
b1 = []
b2 = []
for i in range(len(MK1)):
  b_MK1 = MK1[i][0]
  b_MK2 = MK2[i][0]
  b1.append(b_MK1)
  b2.append(b_MK2)

#ur1 = sorted(ur1)
#ur2 = sorted(ur2)
#b1 = sorted(b1)
#b2 = sorted(b2)

plt.title("BorrowRate vs UtilizationRate")
plt.ylabel("BorrowRate")
plt.xlabel("UtilizationRate")
#plt.xlim((0,1))
#plt.ylim((0,0.2))
plt.scatter(ur1,b1)
plt.scatter(ur2,b2)

In [ ]:
printMarket(MK1)

In [ ]:
printMarket(MK2)

In [ ]:
n_iterations =  4*60*24
prices = []
for i in range(n_iterations):
  prices.append([10,5])

prices[:5]

In [ ]:
n_users = 10
MK1 = marketSetUp(n_users,0.2,0.7,5,0,50,getBorrowAPR_MK1)
MK2 = marketSetUp(n_users,0.15,0.5,10,0,20,getBorrowAPR_MK2)
markets = [MK1,MK2]
collateralFactors = [0.75,0.75]
reserveFactors = [0.2,0.15]
interestModels = [getBorrowAPR_MK1,getBorrowAPR_MK2]

n_iterations =  4*60*24
for i in range(n_iterations):
  markets = newStateMarkets(markets,collateralFactors,i+1,reserveFactors,prices,interestModels,[0,0],[50,50])
  if markets[0][i+1][4]<0 or markets[1][i+1][4]<0:
    print("Desde aquí se jodio la cosa", i)
    break

# plot supply rate
ur1 = []
ur2 = []
for i in range(len(markets[0])):
  ur_MK1 = getUtilizationRate(markets[0],i)
  ur_MK2 = getUtilizationRate(markets[1],i)
  ur1.append(ur_MK1)
  ur2.append(ur_MK2)

# Borrow APR 
b1 = []
b2 = []
for i in range(len(markets[0])):
  b_MK1 = markets[0][i][0]
  b_MK2 = markets[1][i][0]
  b1.append(b_MK1)
  b2.append(b_MK2)

ur1 = sorted(ur1)
ur2 = sorted(ur2)
b1 = sorted(b1)
b2 = sorted(b2)

plt.title("BorrowRate vs UtilizationRate")
plt.ylabel("BorrowRate")
plt.xlabel("UtilizationRate")
#plt.xlim((0,1))
#plt.ylim((0,0.2))
plt.scatter(ur1,b1)
plt.scatter(ur2,b2)

In [ ]:
printMarket(markets[0])
print("###############################################################################3")
printMarket(markets[1])

In [ ]:
# plot supply rate
t1 = []
t2 = []
for i in range(len(MK1)):
  t_MK1 = markets[0][i][5]
  t_MK2 = markets[0][i][5]
  t1.append(t_MK1)
  t2.append(t_MK2)

plt.title("Bloques sin acciones")
plt.ylabel("Cantidad de bloques")
plt.xlabel("bloques")
#plt.ylim(top=1.5)
plt.scatter(range(len(markets[0])),t1)
plt.scatter(range(len(markets[0])),t2)

In [ ]:
# Borrow APR 
s1 = []
s2 = []
for i in range(len(markets[0])):
  s_MK1 = markets[0][i][1]
  s_MK2 = markets[0][i][1]
  s1.append(s_MK1)
  s2.append(s_MK2)

s1 = sorted(s1)
s2 = sorted(s2)

plt.title("SupplyRate vs UtilizationRate")
plt.ylabel("SupplyRate")
plt.xlabel("UtilizationRate")
plt.xlim((0,1))
plt.ylim((0,0.1))
plt.scatter(ur1,s1)
plt.scatter(ur2,s2)

In [ ]:
reserve1 = []
reserve2 = []
k=0
for i in range(len(markets[0])):
  totalReserve1 = markets[0][i][3]
  totalReserve2 = markets[0][i][3]
  reserve1.append(totalReserve1)
  reserve2.append(totalReserve2)
  k += 1

plt.title("Reservas por bloque")
plt.ylabel("Precio de las reservas")
plt.xlabel("bloques")
plt.scatter(range(len(markets[0][:100])),reserve1[:100])
plt.scatter(range(len(markets[0][:100])),reserve2[:100])

In [ ]:
exc1 = []
exc2 = []
k=0
for i in range(len(markets[0])):
  exc1_1 = 1/markets[0][i][4]
  exc2_1 = 1/markets[0][i][4]
  exc1.append(exc1_1)
  exc2.append(exc2_1)
  k += 1

plt.title("Exchange por bloque")
plt.ylabel("Valor de los cTokens")
plt.xlabel("bloques")
plt.plot(range(len(markets[0][:100])),exc1[:100])
plt.plot(range(len(markets[0][:100])),exc2[:100])

In [ ]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import pandas as pd

figsize(16,6)

x = np.linspace(0, 5)
N = ss.norm
USDCoin = N.pdf(x,1,0.1)  # mean = 15; sigma = 4
USDCoin_prices = []

n_iterations = 1000   #4*24*60
for iteration in range(n_iterations):
  price1 = random.choices(x,USDCoin)[0]
  USDCoin_prices.append(price1)

USDCoin = N.pdf(x,1,0.025)
n_iterations = 600   #4*24*60
for iteration in range(n_iterations):
  price1 = random.choices(x,USDCoin)[0]
  USDCoin_prices.append(price1)

#price_MK1 = []
#for i in range(n_iterations):
#  if i % 60 == 0:
#    price_MK1.append(MK1_prices[i])
#    price_MK2.append(MK2_prices[i])

plt.plot(range(1600),USDCoin_prices)
#plt.plot(range(int(n_iterations/60)),price_MK2)
#plt.ylim(top=60)

In [ ]:
Ether = pd.read_excel("Ethereum.xlsx")
EtherPrices = Ether["PRECIO"][426:]
EtherPrices.plot()

In [ ]:
Bitcoin = pd.read_excel("Bitcoin.xlsx")
BitcoinPrices = Bitcoin["PRECIO"][750:]
BitcoinPrices.plot()

In [ ]:
len(BitcoinPrices)

In [ ]:
# Collateral Factors - Uno por cada mercado

collFactor_ETH = 0.75  # Collateral factor para el mercado MK1
collFactor_BTC = 0.75  # Collateral factor para el mercado MK2   !!!!!Estimar!!!!
collFactor_USD = 0.75  # Collateral factor para el mercado MK3

# Lista con los Collateral Factors
collateralFactors = [collFactor_MK1,
                     collFactor_MK2]

# Reserve Factors - Uno por cada mercado

reserveFactor_ETH = 0.20   # Reserve Factor para el mercado MK1
reserveFactor_BTC = 0.15   # Reserve Factor para el mercado MK2    !!!Estimar!!!
reserveFactor_USD = 0.07   # Reserve Factor para el mercado MK3
 
# Lista con los Reserve Factors                    
reserveFactors = [reserveFactor_MK1, 
                  reserveFactor_MK2]

# Exchange rate iniciales

er_price_ETH = 50
er_price_BTC = 50
er_price_USD = 50

n_users = 10
initial_supply_ETH_Rate = 0.1
initial_supply_BTC_Rate = 0.25
initial_supply_USD_Rate = 0.05

ETH = marketSetUp(n_users,reserveFactor_ETH,initial_supply_ETH_Rate,er_price_ETH,Ether["PRECIO"][0])
BTC = marketSetUp(n_users,reserveFactor_BTC,initial_supply_BTC_Rate,er_price_BTC,Bitcoin["PRECIO"][0])
USD = marketSetUp(n_users,reserveFactor_USD,initial_supply_USD_Rate,er_price_USD,USDCoin_prices[0])
printMarket(ETH)
printMarket(BTC)
printMarket(USD)

In [ ]:
BinanceCoin = pd.read_excel("USDCoin.xlsx")
BinanceCoinPrices = BinanceCoin["PRECIO"]
BinanceCoinPrices.plot()

In [ ]:
n_users = 5

MK1 = marketSetUp(n_users,0.2,0.2,50,initialTokenPrices[0])
MK2 = marketSetUp(n_users,0.15,0.3,50,initialTokenPrices[1])
markets = [MK1,MK2]
n_iterations = 4*60*24
for i in range(n_iterations):
  MK1 = newBlockMarket(markets,MK1,collateralFactors,i+1,0.2,MK1_prices)
  MK2 = newBlockMarket(markets,MK2,collateralFactors,i+1,0.15,MK2_prices)
  markets = [MK1,MK2]

In [ ]:
# plot supply rate
ur1 = []
ur2 = []
for i in range(n_iterations):
  ur_MK1 = getUtilizationRate(MK1,i)
  ur_MK2 = getUtilizationRate(MK2,i)
  ur1.append(ur_MK1)
  ur2.append(ur_MK2)

plt.ylabel("utilization rate")
plt.xlabel("bloques")
plt.ylim(top=1.5)
plt.plot(range(n_iterations),ur1)
plt.plot(range(n_iterations),ur2)

In [ ]:
# plot supply rate
b1 = []
b2 = []
for i in range(n_iterations):
  b_MK1 = MK1[i][0]
  b_MK2 = MK2[i][0]
  b1.append(b_MK1)
  b2.append(b_MK2)

plt.ylabel("Borrow rate")
plt.xlabel("bloques")
plt.ylim(top=1.5)
plt.plot(range(n_iterations),b1)
plt.plot(range(n_iterations),b2)

In [ ]:
# plot supply rate
t1 = []
t2 = []
for i in range(n_iterations):
  t_MK1 = MK1[i][5]
  t_MK2 = MK2[i][5]
  t1.append(t_MK1)
  t2.append(t_MK2)

plt.title("Prestamos por bloque")
plt.ylabel("Total prestado")
plt.xlabel("bloques")
#plt.ylim(top=1.5)
plt.scatter(range(n_iterations),t1)
plt.scatter(range(n_iterations),t2)

In [ ]:
# plot borrow rate
variableHistory = []
borrow1 = []
borrow2 = []
util1 = []
util2 = []
k=0
for i in range(n_iterations):
  utilization1 = getUtilizationRate(MK1,k)
  utilization2 = getUtilizationRate(MK2,k) 
  util1.append(utilization1)
  util2.append(utilization2)
  borrow1.append(getBorrowAPR(utilization1))
  borrow2.append(getBorrowAPR(utilization2))
  k += 1

plt.title("BorrowRate vs UtilizationRate")
plt.ylabel("BorrowRate")
plt.xlabel("UtilizationRate")
plt.ylim(top=0.12)
plt.scatter(util1,borrow1)
plt.scatter(util2,borrow2)

In [ ]:
fig = go.Figure() #data=go.Scatter(x=x, y=y, name = "Borrow APR")

#fig.add_trace(go.Scatter(x=x,y=S,name="Supply APR"))
fig.add_trace(go.Scatter(x=util1,y=borrow1,name="Supply APR - simulación"))
fig.add_trace(go.Scatter(x=util2,y=borrow2,name="Borrow APR - simulación"))
fig.update_layout(xaxis_title='Tasa de utilización',
                  yaxis_title='Tasa APR')
fig.show()

In [ ]:
# plot supply rate
variableHistory = []
supply1 = []
supply2 = []
util1 = []
util2 = []
k=0
for i in range(n_iterations):
  utilization1 = getUtilizationRate(MK1,k)
  utilization2 = getUtilizationRate(MK2,k) 
  borrowRate1 = getBorrowAPR(utilization1)
  borrowRate2 = getBorrowAPR(utilization2)
  supplyRate1 = getSupplyAPR(borrowRate1,0.2,utilization1)
  supplyRate2 = getSupplyAPR(borrowRate2,0.15,utilization2)
  util1.append(utilization1)
  util2.append(utilization2)
  supply1.append(supplyRate1)
  supply2.append(supplyRate2)
  k += 1

plt.title("supplyRate vs UtilizationRate")
plt.ylabel("SupplyRate")
plt.xlabel("UtilizationRate")

plt.scatter(util1,supply1)
plt.scatter(util2,supply2)

In [ ]:
# precio de las reservas por bloque
reserve1 = []
reserve2 = []
k=0
for i in range(n_iterations):
  totalReserve1 = totalReserveMarket(MK1,k,0.2)*MK1_prices[k]
  totalReserve2 = totalReserveMarket(MK2,k,0.15)*MK2_prices[k]
  reserve1.append(totalReserve1)
  reserve2.append(totalReserve2)
  k += 1

plt.title("Reservas por bloque")
plt.ylabel("Precio de las reservas")
plt.xlabel("bloques")
plt.scatter(range(n_iterations),reserve1)
plt.scatter(range(n_iterations),reserve2)

In [ ]:
# precio de las reservas por bloque
reserve1 = []
reserve2 = []
k=0
for i in range(n_iterations):
  totalReserve1 = totalReserveMarket(MK1,k,0.2)
  totalReserve2 = totalReserveMarket(MK2,k,0.15)
  reserve1.append(totalReserve1)
  reserve2.append(totalReserve2)
  k += 1

plt.title("Reservas por bloque")
plt.ylabel("Precio de las reservas")
plt.xlabel("bloques")
plt.scatter(range(n_iterations),reserve1)
#plt.scatter(range(n_iterations),reserve2)

In [ ]:
plt.scatter(range(n_iterations),reserve2)

In [ ]:
# plot supply rate
totalBorrow1 = []
totalBorrow2 = []
for i in range(n_iterations):
  totalBorrow1.append(MK1[i][6])
  totalBorrow2.append(MK2[i][6])

plt.title("Prestamos por bloque")
plt.ylabel("Total prestado")
plt.xlabel("bloques")
#plt.scatter(range(n_iterations-10),totalBorrow1[10:])
plt.scatter(range(n_iterations-200),totalBorrow2[200:])

In [ ]:
# plot supply rate
totalBorrow1 = []
totalBorrow2 = []
for i in range(n_iterations):
  totalBorrow_MK1 = MK1[i][6]
  totalBorrow_MK2 = MK2[i][6]
  totalBorrow1.append(totalBorrow_MK1*MK1_prices[i])
  totalBorrow2.append(totalBorrow_MK2*MK2_prices[i])

plt.title("Prestamos por bloque")
plt.ylabel("Total prestado")
plt.xlabel("bloques")
plt.scatter(range(n_iterations-10),totalBorrow1[10:])
plt.scatter(range(n_iterations-10),totalBorrow2[10:])

In [ ]:
ctk1 = []
ctk2 = []
for i in range(n_iterations):
  ctk_MK1 = MK1[i][4]
  ctk_MK2 = MK2[i][4]
  ctk1.append(ctk_MK1)
  ctk2.append(ctk_MK2)

plt.title("Prestamos por bloque")
plt.ylabel("Total prestado")
plt.xlabel("bloques")
plt.scatter(range(n_iterations),ctk1)
plt.scatter(range(n_iterations),ctk2)

In [ ]:
MK1[0][2]

In [ ]:
def amountEstimation(market,blockNumber,mean=500,scale=0.5):
    if 0 <= market[blockNumber][0] < 0.125:
        collBorrow = mean*(1-3*scale)
        loanBorrow = mean*(1+3*scale)
        withdraw = mean*(1+(3/10)*scale)
    elif 0.125 <= market[blockNumber][0] < 0.25:
        collBorrow = mean*(1-2*scale)
        loanBorrow = mean*(1+2*scale)
        withdraw = mean*(1+(2/10)*scale)
    elif 0.25 <= market[blockNumber][0] < 0.5:
        collBorrow = mean*(1-scale)
        loanBorrow = mean*(1+scale)
        withdraw = mean*(1+(1/10)*scale)
    elif 0.5 <= market[blockNumber][0] < 0.75:
        collBorrow = mean*(1+scale)
        loanBorrow = mean*(1-scale)
        withdraw = mean*(1-(1/10)*scale)
    elif 0.75 <= market[blockNumber][0] < 0.875:
        collBorrow = mean*(1+2*scale)
        loanBorrow = mean*(1-2*scale)
        withdraw = mean*(1-(2/10)*scale)
    elif 0.875 <= market[blockNumber][0] <= 1:
        collBorrow = mean*(1+3*scale)
        loanBorrow = mean*(1-3*scale)
        withdraw = mean*(1-(3/10)*scale)
    amountPerAction = [collBorrow,loanBorrow,withdraw]
    for i in range(len(amountPerAction)):
        if amountPerAction[i] <= 0:
            amountPerAction[i] = 0 
    return amountPerAction

amountEstimation(MK1,0)